In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
from collections import Counter
warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline

# 年报-对外投资信息

In [5]:
df_nianbao_touzi_bkp = pd.read_excel(r'../../Data/CompInfo/年报-对外投资信息.xlsx')

In [6]:
df_nianbao_touzi = df_nianbao_touzi_bkp.copy()

In [7]:
df_nianbao_touzi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93369 entries, 0 to 93368
Data columns (total 4 columns):
企业编号    93369 non-null int64
投资金额    1456 non-null float64
投资占比    1456 non-null float64
年报年份    93367 non-null float64
dtypes: float64(3), int64(1)
memory usage: 2.8 MB


In [39]:
df_nianbao_touzi_clean = df_nianbao_touzi.loc[(
    df_nianbao_touzi['投资金额'].notnull())]

agg_func = {
    '投资金额': ['sum', 'min', 'max'],
    '企业编号': ['count']
}
df_nianbao_touzi_group = df_nianbao_touzi_clean.groupby(
    ['企业编号']).agg(agg_func)
df_nianbao_touzi_group.columns = [
    '_'.join(col).strip() for col in df_nianbao_touzi_group.columns.values]
df_nianbao_touzi_group['企业编号'] = df_nianbao_touzi_group.index
df_nianbao_touzi_group.rename(columns={'企业编号_count':'投资次数'}, inplace=True)
df_nianbao_touzi_group.head(n=2)

,投资金额_sum,投资金额_min,投资金额_max,投资次数,企业编号
企业编号,,,,,
1028,3703345.70,11600.0,3208193.70,7,1028
1134,170384.71,3.0,25505.71,56,1134


# 年报-股东股权转让

In [19]:
df_nianbao_gudongguquanzhuanrang_bkp = pd.read_excel(
    r'../../Data/CompInfo/年报-股东股权转让.xlsx')

In [29]:
df_nianbao_gudongguquanzhuanrang = df_nianbao_gudongguquanzhuanrang_bkp.copy()
df_nianbao_gudongguquanzhuanrang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6855 entries, 0 to 6854
Data columns (total 5 columns):
企业编号       6855 non-null int64
变更前股权比例    5010 non-null object
变更后股权比例    5015 non-null object
股权变更日期     5022 non-null object
年报年份       6849 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 267.9+ KB


In [37]:
# df_nianbao_gudongguquanzhuanrang['变更前股权比例'] = df_nianbao_gudongguquanzhuanrang['变更前股权比例'].str.rstrip(
#     '%').astype(float)
df_nianbao_gudongguquanzhuanrang_clean = df_nianbao_gudongguquanzhuanrang.loc[(
    df_nianbao_gudongguquanzhuanrang['变更后股权比例'].notnull())]
df_nianbao_gudongguquanzhuanrang_clean['变更后股权比例'] = df_nianbao_gudongguquanzhuanrang_clean[~df_nianbao_gudongguquanzhuanrang_clean["变更后股权比例"].str.contains(
    '-')]['变更后股权比例'].str.rstrip('%').astype(float)

df_nianbao_gudongguquanzhuanrang_clean['变更股权比例差'] = df_nianbao_gudongguquanzhuanrang_clean['变更后股权比例'] - \
    df_nianbao_gudongguquanzhuanrang_clean['变更前股权比例']
# df_nianbao_gudongguquanzhuanrang_clean.head(n=3)
agg_func = {
    '变更股权比例差': ['sum', 'max', 'min'],
    '变更前股权比例': ['sum'],
    '变更后股权比例': ['sum'],
    '企业编号': ['count']
}
df_nianbao_gudongguquanzhuanrang_clean_new = df_nianbao_gudongguquanzhuanrang_clean.groupby([
                                                                                            '企业编号']).agg(agg_func)
df_nianbao_gudongguquanzhuanrang_clean_new.columns = [
    '_'.join(col) for col in df_nianbao_gudongguquanzhuanrang_clean_new.columns.values]
df_nianbao_gudongguquanzhuanrang_clean_new.columns=['变更股权比例差_sum','变更股权比例差_max','变更股权比例差_min','变更前股权比例_sum','变更后股权比例_sum','变更次数']
df_nianbao_gudongguquanzhuanrang_clean_new['企业编号']=df_nianbao_gudongguquanzhuanrang_clean_new.index
df_nianbao_gudongguquanzhuanrang_clean_new.head(n=3)

,变更股权比例差_sum,变更股权比例差_max,变更股权比例差_min,变更前股权比例_sum,变更后股权比例_sum,变更次数,企业编号
企业编号,,,,,,,
1001,5.24,2.60,-0.80,90.35,95.59,11,1001
1002,-10.91,-10.91,-10.91,61.96,51.05,1,1002
1013,-13.89,1.11,-15.00,43.90,30.01,2,1013
